In [1]:
%load_ext Cython
import numpy as np
from numpy import asfortranarray as Fort

# The following Cython function is Fortran BLAS for Array Multiplication

It takes in zeroed arrays from python's numpy  and outputs the array product as well as the filled-in array of the inputs. This is done as a concept test of the code to then test with numpy's dot product to ascertain if the cython code produces correct output. 

The array's must be fortran-contiguous (so the cython inputs specify that the stride of the arrays are column-major). The output of the array multiplication is also done column major to preserve output. 

In [2]:
%%cython 

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False
#cython: cdivision=True

#Import module: the matrix multiplication module
from scipy.linalg.cython_blas cimport dgemm

from libc.stdlib cimport rand, RAND_MAX, calloc, malloc, free, abort

####################################################################

#Random Number Generator for 
cdef double rand_value(unsigned int MIN, unsigned int MAX) nogil:
    cdef double scaled

    #generate a random number between 0 and 1
    scaled=rand()/<double>RAND_MAX

    #return a random number between the min and the max
    return (MAX*scaled + MIN)

#Fill Arrays
cdef void FILL_ARRAY(double* arr, size_t iter, 
                     unsigned int MIN, unsigned int MAX) nogil:
    cdef Py_ssize_t i
    for i in xrange(iter):
        arr[i]=rand_value(MIN,MAX)
        
#Ouput Arrays (remember that we are in Fortran contiguous - column -major)
#So cycle through the columns and not through the rows when putting into the 
#output cells.
cdef void OUTPUT(double[::1,:] arr_out, double* arr, int rows, int col) nogil:
    cdef Py_ssize_t i,j
    
    for i in xrange(col):
        for j in xrange(rows):
            arr_out[j,i] = arr[j + i*rows]

cpdef void myfunc(double[::1,:] in1, double[::1,:] in2, double[::1,:] out,
                  char* TransA, char* TransB) nogil:
    cdef: 
        double* a 
        double* b 
        double* c
        char* Trans='T'
        char* No_Trans='N'
        int m, n, k, lda, ldb, ldc, col_c
        int row_a, row_b, col_a, col_b
        unsigned int MIN=1, MAX=5
        double alpha, beta
    
    #dimensions of input arrays
    lda = in1.shape[0]
    col_a = in1.shape[1]
    ldb = in2.shape[0]
    col_b = in2.shape[1]
    
    #dimensions of arrays post operation
    if TransA[0]==Trans[0] and TransB[0]==No_Trans[0]:
        m = col_a; n = col_b ; k = lda
    elif TransB[0]==Trans[0] and TransA[0]==No_Trans[0]:
        m = lda; n = ldb ; k = ldb
    elif TransA[0]==Trans[0] and TransB[0]==Trans[0]:
        m = col_a; n = ldb ; k = lda
    else: 
        m = lda; n = col_b ; k = ldb
    
    #dimensions of array c
    ldc = m; col_c = n
    
    a = <double*> malloc(lda*col_a * sizeof(double))
    b = <double*> malloc(ldb*col_b * sizeof(double))    
    c = <double*> calloc(ldc*col_c, sizeof(double))
    
    if a==NULL or b==NULL or c==NULL: abort()
    
    try:
        #fill in arrays
        FILL_ARRAY(a,lda*col_a,MIN,MAX)
        FILL_ARRAY(b,ldb*col_b,MIN,MAX)

        #scalars associated with C = beta*op(A)*op(B) + alpha*C
        alpha = 1.0
        beta = 0.0
    
        dgemm(TransA, TransB, &m, &n, &k, &alpha, a, &lda, b, &ldb, &beta, c, &ldc)
        
        OUTPUT(in1,a,lda,col_a)
        OUTPUT(in2,b,ldb,col_b)
        OUTPUT(out,c,ldc,col_c)
        
    finally:
        free(a)
        free(b)
        free(c)

The following takes in a memory view of the numpy array only (no use of malloc) and works on them directly . 

In [3]:
%%cython

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False
#cython: cdivision=True

#Import module: the matrix multiplication module
from scipy.linalg.cython_blas cimport dgemm

####################################################################

cpdef void MM(double[::1,:] a, double[::1,:] b, 
              double[::1,:] out, char* TransA, char* TransB) nogil:
    
    cdef:
        char* Trans='T'
        char* No_Trans='N'
        int m, n, k, lda, ldb, ldc
        int col_a, col_b
        double alpha, beta
    
    #dimensions of input arrays
    lda = a.shape[0]
    col_a = a.shape[1]
    ldb = b.shape[0]
    col_b = b.shape[1]
    
    #dimensions of arrays post operation (after transposing, or not)
    if TransA[0]==Trans[0] and TransB[0]==No_Trans[0]:
        m = col_a; n = col_b ; k = lda
    elif TransB[0]==Trans[0] and TransA[0]==No_Trans[0]:
        m = lda; n = ldb ; k = col_a
    elif TransA[0]==Trans[0] and TransB[0]==Trans[0]:
        m = col_a; n = ldb ; k = lda
    else: 
        m = lda; n = col_b ; k = ldb
    
    #leading dimension of c from above
    ldc = m
    
    #scalars associated with C = beta*op(A)*op(B) + alpha*C
    alpha = 1.0
    beta = 0.0
    
    #Fortran BLAS function for calculating the multiplication of arrays
    dgemm(TransA, TransB, &m, &n, &k, &alpha, &a[0,0], &lda, &b[0,0], &ldb, &beta, &out[0,0], &ldc)


In [4]:
n=3
m=3
k=6

test_a = Fort(np.random.randint(2,size=(m,k)), dtype='d')
test_b = Fort(np.random.randint(2,size=(k,n)), dtype='d')
test_c = np.zeros((k,k), dtype='d', order='F')

In [7]:
%timeit MM(test_a,test_b,test_c,b"T",b"T")

The slowest run took 5.87 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.06 µs per loop


In [6]:
%timeit test_a.T.dot(test_b.T)

The slowest run took 20.92 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.74 µs per loop


# Cython code for the inverse of a square array with Fortran LAPACK

This code takes in a square array and rewrites the array as the inverse of itself. It uses dgecon to test that the array is invertible (full rank). If the array is of full rank, then the input array is overwritten; if the array is singular then the array is not overwritten and an output statement to the effect is given.

In [8]:
%%cython

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False
#cython: cdivision=True

#Import module: the matrix multiplication module
from scipy.linalg.cython_lapack cimport dgetrf, dgetri, dgecon, dlacpy, dlange

from libc.stdlib cimport malloc, free, abort

####################################################################
cdef void OUTPUT(double[::1,:] arr_out, double* arr, int rows, int col) nogil:
    
    cdef Py_ssize_t i,j
    
    for i in xrange(col):
        for j in xrange(rows):
            arr_out[j,i] = arr[j + i*rows]

cpdef void INV_M(double[::1,:] a) nogil:
    cdef:
        #array pointers
        int* pivot
        int* IWORK
        double* a_copy
        double* work
        double* work_dgecon
        
        #variables characterizing the arrays
        int m, n, lda, INFO, Lwork
        double ANORM, RCOND, sing_tol = 1e-6
        
        #setting for the functions
        char* NORM = 'I' #The infinity norm (consistent use between dlange & dgecon)
        char* UPLO = 'O' #Any letter other then 'U' or 'L' will copy entire array
    
    #Dimensions of arrays
    m = a.shape[0]
    n = a.shape[1]
    lda = m
    Lwork = m**2

    #manually allocate memory
    #Note: 'work' can be used by both dlange and dgetri as its construction is the same
    pivot = <int*> malloc(m * sizeof(int))
    IWORK = <int*> malloc(n * sizeof(int))
    a_copy = <double*> malloc(m*n * sizeof(double))
    work = <double*> malloc(Lwork * sizeof(double))
    work_dgecon = <double*> malloc(4*n * sizeof(double))
    
    if (pivot==NULL or IWORK==NULL or a_copy==NULL 
        or work==NULL or work_dgecon==NULL): 
            abort()
    
    try:
        #First, create a copy of the array to invert
        dlacpy(UPLO, &m, &n, &a[0,0], &lda, a_copy, &lda)
        
        #Next, compute the NORM(a) on the a_copy to preserve array a
        ANORM = dlange(NORM, &m, &n, a_copy, &lda, work)
        
        #Conduct the LU factorization of the array a
        dgetrf(&m, &n, a_copy, &lda, pivot, &INFO)
        
        #Check that LU factorization was successful:
        if INFO==0:
        
            #Now use dgecon to check that the array is invertible (non-singular)
            dgecon(NORM, &n, a_copy, &lda, &ANORM, &RCOND, work_dgecon, IWORK, &INFO)
            
            if RCOND > sing_tol:
       
                #Now use the LU factorization and the pivot information to invert
                dgetri(&n, a_copy, &lda, pivot, work, &Lwork, &INFO)
                
                OUTPUT(a,a_copy,m,n)
            
            else: 
                with gil:
                    print("Array is singular and will not be inverted")
            
        else: 
            with gil:
                print("The factor U is singular")
        
    finally:
        free(pivot)
        free(work)
        free(work_dgecon)
        free(IWORK)
        free(a_copy)

Test the Cython functions for matrix multiplication and inversion

In [50]:
n=4
m=4
k=3

a = np.zeros((m,k), dtype='d', order='F')
b = np.zeros((k,n), dtype='d', order='F')
c = np.zeros((m,n), dtype='d', order='F')

In [51]:
myfunc(a,b,c,b'N',b'N')

In [52]:
print(c)

[[ 30.46644183  23.29393259  33.51062539  33.25664379]
 [ 17.40451611  20.67289111  20.83262244  26.91747411]
 [ 43.12236564  39.88345658  49.54803621  54.60530938]
 [ 27.52262801  24.96112473  32.2681151   34.4646216 ]]


In [53]:
print(a.dot(b))

[[ 30.46644183  23.29393259  33.51062539  33.25664379]
 [ 17.40451611  20.67289111  20.83262244  26.91747411]
 [ 43.12236564  39.88345658  49.54803621  54.60530938]
 [ 27.52262801  24.96112473  32.2681151   34.4646216 ]]


In [54]:
INV_M(c)

Array is singulary and will not be inverted


In [55]:
print(c) #Because the array is deemed singular, it is not overwritten!!

[[ 30.46644183  23.29393259  33.51062539  33.25664379]
 [ 17.40451611  20.67289111  20.83262244  26.91747411]
 [ 43.12236564  39.88345658  49.54803621  54.60530938]
 [ 27.52262801  24.96112473  32.2681151   34.4646216 ]]


In [56]:
print (np.linalg.inv(a.dot(b)))

[[ -1.50779259e+13  -1.74356438e+13   2.56124361e+13  -1.24130277e+13]
 [ -1.66706541e+14  -1.92774250e+14   2.83179575e+14  -1.37242544e+14]
 [ -3.08200977e+13  -3.56394009e+13   5.23532078e+13  -2.53729014e+13]
 [  1.61634543e+14   1.86909150e+14  -2.74563919e+14   1.33066980e+14]]


The above shows that the Cython function can catch a singular array and prevent the work of inverting it, whereas NUMPY will invert the array even though it is clearly singular. In theory, the LU decomposition should not occur if the array is singular. However, LU decomposition can still occur because of machine precision on what '0' is. So using DGECON Fortran subroutine can get us (to a tolerance) a condition of rank, and prevent the inversion if we do not meet it. 

Now to test if the Cython function is faster than numpy.

In [31]:
d = np.random.randint(5,size=(1000,1000)).astype('d')

In [32]:
np.linalg.matrix_rank(d)

1000

In [33]:
e = Fort(d.copy(), dtype='d')
f = Fort(d.copy(), dtype='d')

In [34]:
%timeit INV_M(e)

10 loops, best of 3: 29.9 ms per loop


In [35]:
%timeit np.linalg.inv(d)

10 loops, best of 3: 34.2 ms per loop


In [38]:
%timeit np.linalg.inv(f)

10 loops, best of 3: 32.2 ms per loop


For the Cython test the array MUST be Fortran contiguous. For the Numpy test we can see that a fortran contiguous array is faster for inverting than a C contiguous (likely because numpy is making Fortran calls); however it is slower than the Cython function. 

# Test code in Cython for making a copy of an array using LAPACK routines

In [39]:
%%cython

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False
#cython: cdivision=True

#Import module: the matrix multiplication module
from scipy.linalg.cython_lapack cimport dlacpy
from libc.stdlib cimport malloc, free, abort

cdef void OUTPUT(double[::1,:] arr_out, double* arr, int rows, int col) nogil:
    cdef Py_ssize_t i,j
    
    for i in xrange(col):
        for j in xrange(rows):
            arr_out[j,i] = arr[j + i*rows]

def COPY(double[::1,:] a, double[::1,:] b, char* UPLO):
    
    cdef:
        double* a_copy
        int m, n, lda, INFO
   
    #Dimensions of arrays
    m = a.shape[0]
    n = a.shape[1]
    lda = m
    
    a_copy = <double*> malloc(m*n * sizeof(double))
    
    if a_copy==NULL: abort()

    #First, create a copy of the array to invert
    dlacpy(UPLO, &m, &n, &a[0,0], &lda, a_copy, &lda)
    
    OUTPUT(b, a_copy, m, n)
    
    free(a_copy)


In [40]:
f = Fort(np.random.randint(5,size=(4,4)), dtype='d')
g = Fort(np.empty_like(f), dtype='d')
g.fill(np.nan)
print(f)

[[ 1.  4.  1.  1.]
 [ 0.  4.  2.  4.]
 [ 2.  4.  3.  3.]
 [ 2.  3.  0.  1.]]


In [41]:
COPY(f,g,b'O')

In [42]:
g

array([[ 1.,  4.,  1.,  1.],
       [ 0.,  4.,  2.,  4.],
       [ 2.,  4.,  3.,  3.],
       [ 2.,  3.,  0.,  1.]])